In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

from scipy.special import expit

- - -

script para converter a planilha de pesos em um dicionário javascript:

In [337]:
df = pd.read_csv('data/weights.csv')

In [338]:
d = dict()

QUESTION = str()
INDEX    = int()

for i,r in df.iterrows():
    if str(r['question']) != 'nan':
        INDEX    = str(i).zfill(3)
        d[INDEX] = dict()
        d[INDEX]['question']   = r['question']
        d[INDEX]['answers'] = [ [r['answer'], [float(r['w0']),
                                                 float(r['w1']),
                                                 float(r['w2']),
                                                 float(r['w3']),
                                                 float(r['w4'])] 
                                   ] 
                                 ]
    else:
        d[INDEX]['answers'].append( [r['answer'], [float(r['w0']),
                                                 float(r['w1']),
                                                 float(r['w2']),
                                                 float(r['w3']),
                                                 float(r['w4'])] 
                                   ]  )

In [339]:
with open('quizquestions.js', 'w') as qq:
    for k in sorted(d):
        qq.write(
"""
{{
      question: "{}",
      answers: {{
""".format(d[k]['question']))

                
        for i,v in enumerate(d[k]['answers']):
            if i < len(d[k]['answers'])-1:
                qq.write(
"""
        {}:"{}",
""".format(str(i), v[0]))
            else:
                qq.write(
"""
        {}:"{}" }},
""".format(str(i), v[0]))
        
        qq.write(
"""
      weights: [
""")
                
        for i,v in enumerate(d[k]['answers']):
            if i < len(d[k]['answers'])-1:
                qq.write(
"""
                    {},
""".format(repr(v[1])))
            else:
                qq.write(
"""
                    {} ],
      img: "pig"
    }},
""".format(repr(v[1])))


- - -

transformando os pesos em tensor:

In [174]:
max_answer_shape = 0
# get max_answer_shape
for k in sorted(d):
    q = d[k]
    if len(q['answers']) > max_answer_shape:
        max_answer_shape = len(q['answers'])

T = np.multiply.outer(np.multiply.outer(np.zeros(len(d)), np.zeros(max_answer_shape)), np.zeros(5))

In [175]:
T.shape

(13, 5, 5)

In [176]:
index = 0
for k in sorted(d):
    q = d[k]
    for i,v in enumerate(q['answers']):
        T[index][i] = v[1]
    
    index += 1

In [177]:
pkl.dump( T, open( "B.p", "wb" ) )

- - -

convertendo questionário em dataframe

In [5]:
df = pd.read_csv('data/weights_calibration.csv')

In [7]:
DATA     = dict()
PROFILES = 4
max_answer_shape = 5

for i in range(PROFILES):
    COL = 'answers_{}'.format(str(i))
    
    d         = dict()
    DATA[COL] = dict()
    QUESTION = str()
    INDEX    = int()
    ANS_ID   = 0
    
    for i,r in df.iterrows():
        if str(r['question']) != 'nan':
            INDEX    = str(i).zfill(3)
            ANS_ID   = 0
            
            if str(r['question']) != 'TARGET':            
                d[INDEX] = dict()
                d[INDEX]['question']   = r['question']
                d[INDEX]['answers']    = np.zeros(max_answer_shape)
                if str(r[COL]) != 'nan':
                    d[INDEX]['answers'][ANS_ID] = 1

                ANS_ID += 1
            else:
                d['TARGET'] = np.zeros(max_answer_shape)
                d['TARGET'][int(r[COL])] = 1
        else:
            if str(r[COL]) != 'nan':
                d[INDEX]['answers'][ANS_ID] = 1
            
            ANS_ID += 1

    for i,k in enumerate(sorted(d)):
        if k != "TARGET":
            DATA[COL]['ans_{}'.format(str(i).zfill(2))] = d[k]['answers']
        else:
            DATA[COL]['TARGET'] = d[k]

In [8]:
data = pd.DataFrame.from_dict(DATA, orient='index')

In [9]:
df = data[
['ans_00',
'ans_01',
'ans_02',
'ans_03',
'ans_04',
'ans_05',
'ans_06',
'ans_07',
'ans_08',
'ans_09',
'ans_10',
'ans_11',
'ans_12',
'TARGET']]

In [10]:
df

,ans_00,ans_01,ans_02,ans_03,ans_04,ans_05,ans_06,ans_07,ans_08,ans_09,ans_10,ans_11,ans_12,TARGET
answers_0,"[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 0.0, 1.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]"
answers_1,"[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 0.0, 1.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]"
answers_2,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]"
answers_3,"[0.0, 1.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]"


- - -

backpropagation de pobre:

In [11]:
max_answer_shape = 5
Q                = 14
WEIGHTS          = pkl.load( open( "B.p", "rb" ) )
ANS              = ['ans_00',
                    'ans_01',
                    'ans_02',
                    'ans_03',
                    'ans_04',
                    'ans_05',
                    'ans_06',
                    'ans_07',
                    'ans_08',
                    'ans_09',
                    'ans_10',
                    'ans_11',
                    'ans_12']
eta = 0.015

def back(_df):
    
    A = _df[ ANS ].T.values
    y = _df[ 'TARGET' ]
    
    A = np.stack(A, axis=0)
    B = np.copy(WEIGHTS)
    C = np.zeros( (len(A),5) )
    
    for i in range(len(A)):
        C[i] = np.sum(B[i]*A[i].T[:, np.newaxis], axis=0)
    
    D = np.sum(C, axis=0)
    
    #
    
    W = np.copy(WEIGHTS)
    
    for ITERATIONS in range(250):
        B = np.copy(W)
        for i in range(len(A)):
            C[i] = np.sum(B[i]*A[i].T[:, np.newaxis], axis=0)
        D = np.sum(C, axis=0)
        
        dW = 0*np.copy(WEIGHTS)
        
        for I in range(len(dW)):
            for J in range(len(dW[0])):
                for K in range(len(dW[0][0])):
                    OMEGA = (D[K] - y[K] )
                    dL    = 2 * OMEGA * A[I][J]

                    dW[I][J][K] = dL
        W = np.copy(W -eta*dW)
    
    return W

In [12]:
avg_W = [ back(df.iloc[i]) for i in range(df.shape[0]) ]
avg_W = avg_W + WEIGHTS

In [16]:
Wstar = sum(avg_W)/len(avg_W)